In [12]:
import os
from groq import Groq
import dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

In [2]:
dotenv.load_dotenv('.env') 
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

# create voi search bang L2

In [5]:
texts = [
    "Fast language models improve efficiency in AI applications.",
    "They enable real-time interactions and better user experience.",
    "Optimized models reduce computational costs and energy consumption.",
    "Cat and dog are animals",
]

# Create a document for each text
documents = [Document(page_content=text, metadata={'source': 'test'}) for text in texts]

In [6]:
# Khởi tạo vector database với FAISS
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'}
)

In [10]:
index = faiss.IndexFlatL2(len(embeddings.embed_query('thang')))

In [13]:
vector_store = FAISS(
    embedding_function=embeddings,
    index= index ,
    index_to_docstore_id={},
    docstore=InMemoryDocstore()
)

In [19]:
vector_store.add_documents(documents) 

['3332b1d9-b529-48e9-a48b-9f6455e64f06',
 '5ac71e67-b9a2-4a64-a766-1d34d1c886d5',
 'd2bda9f1-449a-4a18-bd51-a9ec8240ff67',
 '44c9d655-1202-43fb-ab31-e055db88165c']

In [20]:
results = vector_store.similarity_search(
    "Cat and dog are animals",
    k=1,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

# create voi search cua langchain

In [ ]:
vector_db = FAISS.from_texts(texts, embeddings)
FAISS.save_local(vector_db, "test_faiss")
print(vector_db)

In [10]:
vector_db = FAISS.load_local("test_faiss", embeddings, allow_dangerous_deserialization=True)
print(vector_db.index.ntotal)

4


In [ ]:

# search similar documents
def retrieve_relevant_documents(query, k=10):
    # Lấy kết quả từ Vector Database
    docs= vector_db.similarity_search(query, k=k)
    
    results = [doc.page_content for doc in docs]
    
    return results

from sentence_transformers import CrossEncoder


# rerank search results
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def retrieve_and_re_rank_advanced(vector_db, query, k=10):
    # Lấy kết quả từ Vector Database
    docs_with_scores = vector_db.similarity_search_with_score(query, k=k)
    
    # Chuẩn bị dữ liệu cho Cross-Encoder
    doc_texts = [doc.page_content for doc, _ in docs_with_scores]
    pairs = [[query, doc] for doc in doc_texts]
    
    # Dùng Cross-Encoder để đánh giá lại
    rerank_scores = cross_encoder.predict(pairs)
    
    # Sắp xếp lại theo score mới
    ranked_docs = sorted(zip(doc_texts, rerank_scores), key=lambda x: x[1], reverse=True)
    
    results = [doc for doc, _ in ranked_docs]
    scores = [score for _, score in ranked_docs]
    
    return results, scores


In [ ]:
# input query
query = "Cat and dog are what?"
retrieved_docs = retrieve_relevant_documents(query, 1)
print(retrieved_docs)

Cat and dog are animals


In [13]:
# Gửi yêu cầu đến mô hình Groq
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant that utilizes retrieved information."},
        {"role": "user", "content": f"Context:\n{retrieved_docs}\n\nQuestion: {query}"},
    ],
    model="llama-3.3-70b-versatile",
)

print("Question: ",chat_completion.choices[0].message.content)

Question:  Cat and dog are animals.


## tao vector db voi pdf

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

In [5]:
pdf_path = 'pdf'
vector_db_path = 'vector_db_pdf'

In [7]:
loader = DirectoryLoader(
    path = pdf_path,
    glob = "*.pdf",
    loader_cls = PyPDFLoader
)
documents = loader.load()

In [ ]:
print(len(documents))
## duoc chia thanh cac document nho

65


In [22]:
print(documents[1].page_content)

Mục lục
1 Giới thiệu . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
2 Phương pháp . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
2.1 Mô tả bài toán . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
2.2 Dữ liệu . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
2.3 Quy trình xử lí dữ liệu . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
2.4 Quy trình tạo Agent . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
2.5 Xây dựng ứng dụng . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7
2.6 Quy trình đánh giá hệ thống . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8
3 Các thành phần của Llammaindex . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9
3.1 Yêu cầu trước khi bắt đầu . . . . . . . . . . . . . . . . . . . . . .

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=512,
    chunk_overlap=64,
    length_function=len
)

In [24]:
chunks = text_splitter.split_documents(documents)

In [31]:
chunks_text = [chunk.page_content for chunk in chunks]

In [32]:
db_pdf = FAISS.from_texts(chunks_text, embeddings)
FAISS.save_local(db_pdf, vector_db_path)


In [33]:
db_pdf = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)

In [61]:
query = "AI agent la gi?"
retrieved_docs2, score = retrieve_and_re_rank_advanced(db_pdf, query)
for doc, s in zip(retrieved_docs2, score):
    print(doc)
    print(s)
    print('----------' * 100)
print('----------' * 100)


AIO 2024 aivietnam.edu.vn
7.4 Agent
Nếu chúng ta hỏi chatbot "Mấy giờ rồi nhỉ?", hay ta muốn chatbot viết một bức thư rồi gửi nó qua địa
chỉ email của khách hàng, hoặc ta có một cơ sở dữ liệu muốn chatbot có thể kết nối và truy xuất thông
tin dựa trên yêu cầu, sau đó viết thành file báo cáo rồi gửi cho ta, giống như một nhân viên thật sự?...
nói chung là chúng ta muốn chatbot có thể thực hiện nhiều công việc hơn thì làm như thế nào?
1.4298093
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [62]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant that utilizes retrieved information."},
        {"role": "user", "content": f"Context:\n{retrieved_docs2[0]}\n\nQuestion: {query}"},
    ],
    model="llama-3.3-70b-versatile",
)

print("Question: ",chat_completion.choices[0].message.content)

Question:  Một câu hỏi hay!

Theo như context bạn cung cấp, có vẻ như bạn đang tìm hiểu về công nghệ AI và ứng dụng của nó trong việc tạo ra các chatbot thông minh.

Một Agent (đại lý) trong lĩnh vực AI là một chương trình máy tính có thể thực hiện các công việc tự động dựa trên yêu cầu của người dùng. Agent có thể hiểu và thực hiện các nhiệm vụ phức tạp, như trả lời câu hỏi, thực hiện các hành động, hoặc thậm chí là tương tác với các hệ thống khác.

Trong trường hợp của chatbot, một Agent có thể được thiết kế để thực hiện các công việc như:

* Trả lời câu hỏi của người dùng
* Tìm kiếm và cung cấp thông tin
* Thực hiện các hành động như gửi email hoặc tạo file báo cáo
* Tương tác với các hệ thống khác để thu thập hoặc cập nhật thông tin

Để tạo ra một Agent như vậy, chúng ta cần sử dụng các công nghệ như:

* Natural Language Processing (NLP) để hiểu và xử lý ngôn ngữ tự nhiên
* Machine Learning (ML) để huấn luyện Agent học hỏi và cải thiện
* Integration với các hệ thống khác để thực hi